In [1]:
import numpy as np

import zarr_io
import data_prep
import coarse_registration, fine_registration

from sofima import stitch_rigid

In [2]:
bucket = 'aind-open-data'
downsampling_exp = 2
tile_volumes = []  # Filled with x in outer loop and y in inner loop
for x in range(5):
    for y in range(3):
        dataset = 'exaSPIM_615296_2022-09-28_11-47-06/exaSPIM/'
        path = f'tile_x_000{x}_y_000{y}_z_0000_ch_488/{downsampling_exp}'
        tile = zarr_io.open_zarr_s3(bucket, dataset + path)
        tile_volumes.append(tile.T[:,:,:,0,0])

# Inferred from Neuroglancer
tile_layout = np.array([[14, 11, 8, 5, 2], 
                        [13, 10, 7, 4, 1], 
                        [12,  9, 6, 3, 0]])

In [5]:
tile_layout.shape

(3, 5)

In [3]:
cx, cy = coarse_registration.compute_coarse_offsets(tile_layout, tile_volumes)
coarse_mesh = stitch_rigid.optimize_coarse_mesh(cx, cy, mesh_fn=stitch_rigid.elastic_tile_mesh_3d)

print(f'{cx=}')
print(f'{cy=}')

Left Id: 14, Right Id: 11
Left: (0, 0), Right: (0, 1) [ 6.249e+03 -1.000e+00 -1.000e+02]
Left Id: 11, Right Id: 8
Left: (0, 1), Right: (0, 2) [ 6.249e+03 -1.000e+00 -1.000e+02]
Left Id: 8, Right Id: 5
Left: (0, 2), Right: (0, 3) [ 6.249e+03 -1.000e+00 -1.000e+02]
Left Id: 5, Right Id: 2
Left: (0, 3), Right: (0, 4) [ 6.249e+03 -1.000e+00 -2.300e+01]
Left Id: 13, Right Id: 10
Left: (1, 0), Right: (1, 1) [ 6.249e+03 -1.000e+00 -1.000e+02]
Left Id: 10, Right Id: 7
Left: (1, 1), Right: (1, 2) [ 6.249e+03 -1.000e+00 -1.000e+02]
Left Id: 7, Right Id: 4
Left: (1, 2), Right: (1, 3) [ 6.249e+03 -1.000e+00 -1.000e+02]
Left Id: 4, Right Id: 1
Left: (1, 3), Right: (1, 4) [ 6.249e+03 -1.000e+00 -1.000e+02]
Left Id: 12, Right Id: 9
Left: (2, 0), Right: (2, 1) [ 6.249e+03 -1.000e+00 -1.000e+02]
Left Id: 9, Right Id: 6
Left: (2, 1), Right: (2, 2) [ 6.249e+03 -1.000e+00 -1.000e+02]
Left Id: 6, Right Id: 3
Left: (2, 2), Right: (2, 3) [ 6.249e+03 -1.000e+00 -1.000e+02]
Left Id: 3, Right Id: 0
Left: (2, 3)

In [4]:
cx.shape

(3, 1, 3, 5)

In [4]:
# Update tile volumes -- need a 1zyx shape
bucket = 'aind-open-data'
downsampling_exp = 2
tile_volumes = []  # Filled with x in outer loop and y in inner loop
for x in range(5):
    for y in range(3):
        dataset = 'exaSPIM_615296_2022-09-28_11-47-06/exaSPIM/'
        path = f'tile_x_000{x}_y_000{y}_z_0000_ch_488/{downsampling_exp}'
        tile = zarr_io.open_zarr_s3(bucket, dataset + path)
        tile_volumes.append(tile[0,:,:,:,:])

# Inferred from Neuroglancer
tile_layout = np.array([[14, 11, 8, 5, 2], 
                        [13, 10, 7, 4, 1], 
                        [12,  9, 6, 3, 0]])

In [5]:
# Fine Registration, compute patch flows
stride = 20, 20, 20
tile_size_xyz = tile_volumes[0].T[0:3]
flow_x, offsets_x = fine_registration.compute_flow_map3d(tile_layout, tile_volumes,
                                                        tile_size_xyz, cx, axis=0,
                                                        stride=stride,
                                                        patch_size=(80, 80, 80))

flow_y, offsets_y = fine_registration.compute_flow_map3d(tile_layout, tile_volumes,
                                                        tile_size_xyz, cy, axis=1,
                                                        stride=stride,
                                                        patch_size=(80, 80, 80))

# We'll pick it up from here tomorrow.

AttributeError: 'NoneType' object has no attribute 'start'

In [ ]:
# Going to simplify/standardize the elastic registration input to be identical to the coarse registration (as it should)
# Implictly will make xyz the standard image shape, and operations that require the opposite will be performed internal to function call. 


